# **Análisis Discriminante lineal**

El objetivo del Análisis Discriminante lineal (LDA) es encontrar el subespacio de características que optimiza la separabilidad de las clases y es una técnica supervisada.
El LDA como se conoce actualmente para múltiples clases, funciona bajo la suposición de que las clases tienen matrices de covarianza iguales y las clases tienen distribuciones normales. Otra suposición es que las instancias en el conjunto de entrenamiento son independientes entre si. Si una o más de una de las suposiciones anteriores no se cumplieran, la técnica del LDA funciona bastante bien.

## Pasos en el análisis discriminante lineal



Sea un conjunto de datos $\mathbf{X}$ de dimensiones $n\times d$ de $d$ características y n instancias:



1.   Normalizar los datos $d$-dimensionales.
2.   Calcular los vectores de medias $\mathbf{m}_i$ para cada clase.
$$\mathbf{m}_i=\frac{1}{n_i}\sum_{\mathbf{x}\in D_i} \mathbf{x}_m$$
3.   Calcular las matrices: matriz de dispersión entre clases $\mathbf{S}_B$ y matriz de dispersión intra clase $\mathbf{S}_W$.
4.   Calcular los autovectores y sus correspondientes autovalores de la matriz $\mathbf{S}^{-1}_W \mathbf{S}_B$.
5.   Ordenar los autovectores en orden decreciente a los autovalores y Seleccionar $k$ autovectores que correspondan a los $k$ autovalores más grandes.
6.   Formar una matriz $\mathbf{W}$ de dimensiones $d \times k$ donde cada columna representa un autovector.
7.   Proyectar las muestras en el nuevo subespacio de características usando la matriz de transformación para obtener los datos transformados en dimensión $n \times k$.





In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
from IPython.display import Image, display
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    %cd '/content/drive/MyDrive/Inteligencia Artificial/IA - Clases de Práctica/ContenidosPorTemas'

Vamos a utilizar un dataset disponible de vinos (Wine dataset) que consiste en 178 instancias de 13 características que describen sus propiedades químicas. Las instancias pertenecen a una de 3 clases: 1,2,3 que hacen referencia a 3 tipos uvas cultivadas en una misma región de Italia pero derivadas de diferentes viñedos.

In [ ]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine

Extrae del dataset la columna correspondiente a las clases (columna 0) para formar el vector **y** y los datos **X** correspondientes a las características

Divide los datos en conjuntos de entrenamiento `X_train` y `y_train` y conjuntos de prueba `X_test` y `y_test` utiliza `random_state=42`

In [ ]:
from sklearn.model_selection import train_test_split



1. Normaliza los datos (d=13) para obtener las variables escaladas `X_train_std` y `X_test_std`.

In [ ]:
from sklearn.preprocessing import StandardScaler


2. Calcular los vectores de medias $\mathbf{m}_i$ para cada clase.
$$\mathbf{m}_i= \left[ \begin{align} &\mu_0^{i} \\ &\mu_1^{i} \\ &\:\vdots \\ &\mu_{11}^{i} \end{align} \right] , \quad i=1,2,3$$

Guardar los 3 vectores en una lista llamada `mean_vecs`

In [ ]:
np.set_printoptions(precision = 4)



3. Cálculo de las matrices:

**Matriz intra-clase:**

$\mathbf{S}_W = \sum \mathbf{S_i}$

Matriz de dispersión por clase:

$\mathbf{S}_i = \sum_{\mathbf{x}\in D_i}(\mathbf{x} - \mathbf{m}_i)(\mathbf{x}- \mathbf{m}_i)^T$

donde $D_i$ es el conjunto de intancias correspondiente a la clase $i$

In [ ]:
d = 13
S_W = np.zeros((d, d))
for label, mv in zip(range(1, 4), mean_vecs):
    class_scatter = np.zeros((d, d))  # matriz de dispersión de cada clase
    for row in X_train_std[y_train == label]:
        row, mv = row.reshape(d, 1), mv.reshape(d, 1)  # vectores columna
        class_scatter += (row - mv).dot((row - mv).T)
    S_W += class_scatter                          # suma de las matrices de dispersión

print('Matriz de dispersión intra-clase: '
      f'{S_W.shape[0]}x{S_W.shape[1]}')

**Matriz entre-clases:**

$\mathbf{S}_B = \sum_{i=1}^{c}n_i(\mathbf{m}_i - \mathbf{m})(\mathbf{m}_i- \mathbf{m})^T$

Donde $\mathbf{m}$ es el vector de medias global considerando todas las clases.

In [ ]:
mean_overall = np.mean(X_train_std, axis=0)
mean_overall = mean_overall.reshape(d, 1)  # lo hago vector columna

d = 13  # number of features
S_B = np.zeros((d, d))

for i, mean_vec in enumerate(mean_vecs):
    n = X_train_std[y_train == i + 1, :].shape[0]
    mean_vec = mean_vec.reshape(d, 1)
    S_B += n * (mean_vec - mean_overall).dot((mean_vec - mean_overall).T)

print('Matriz de dispersión entre clases: '
      f'{S_B.shape[0]}x{S_B.shape[1]}')

4.   Calcule los autovectores y sus correspondientes autovalores de la matriz $\mathbf{S}^{-1}_W \mathbf{S}_B$.

In [ ]:
eigen_vals, eigen_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))

5.   Ordene los autovectores en orden decreciente a los autovalores y Seleccione $k$ autovectores de acuerdo a los $k$ autovalores más grandes

**Ordene los autovectores en orden descendente de los autovalores:**

In [ ]:
# lista de tuplas (autovalor, autovector)
eigen_pairs = [ (np.abs(eigen_vals[i]), eigen_vecs[:, i].real) for i in range(len(eigen_vals))]

# ordenamos las tuplas de mayor a menor
eigen_pairs = sorted(eigen_pairs, key=lambda k: k[0], reverse=True)
print('Autovalores en orden descendente:\n')
for eigen_val in eigen_pairs:
    print(eigen_val[0])

6.   Forme la matriz $\mathbf{W}$ de dimensiones $d \times k$ donde cada columna representa un autovector.

In [ ]:
w = np.hstack( (eigen_pairs[0][1].reshape(13,1), eigen_pairs[1][1].reshape(13,1)) )
print('Matriz W:\n', w)

7.   Proyecte las muestras en el nuevo subespacio de características usando la matriz de transformación para obtener los datos transformados en dimensión $n \times k$.

In [ ]:
X_train_lda = X_train_std.dot(w)

Realice un **gráfico de dispersión** de los datos tranformados `X_train_lda` en 2 dimensiones, utilice diferentes marcadores ('o', 's' ,'^') y diferentes colores para las 3 clases. Considera que las clases son separables en este subespacio?

In [ ]:
colors = ['r', 'b', 'g']
markers = ['o', 's', '^']



## LDA usando Scikit-Learn

Usando la [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html), utilice el análisis discriminante lineal de scikit learn sobre `X_train_std`para obtener el mismo resultado. Compare los graficos de dispersión.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

#Entrenamiento

In [ ]:
#Grafico los datos transformados (gráfico de dispersión)

Evalue el modelo con los datos de prueba `X_test_std` y calcule el `accuracy`